# Root Finding: Bisection Method
---

GENERAL PROBLEM: find the real roots of a given function $f(x)$ in the case when closed-form solutions are not available. That is, find the values of $x$ that satisfy the equation

\begin{align}
  f(x) = 0
\end{align}

where $x$ is a real variable, and $f(x)$ is some non-linear function.

IDEA: start with an interval where the root is known to exist, divide (bisect) the interval into two subintervals, performing a simple test to determine which subinterval the root is in, and repeat until the desired precision is obtained. 

PRE-REQUISITES:
- [None]

REFERENCES:
- [1] DeVries and Hasbun, *A First Course in Computational Physics, 2nd edition*.
- [2] Burden and Faires, *Numerical Analysis, 7th edition*.
- [3] Ralston and Rabinowitz, *A First Course in Numerical Analysis, 2nd edition*.

## 1. Algorithm

**Step 0.** Identify an interval $[a,b]$ where the root in question is known to exist (and no other roots exist!). Set $a_{0}=a$ and $b_{0}=b$. Also, choose the desired relative error as the tolerance that the answer is required to have. 

**Step 1.** Divide the interval in half. Call this midpoint $c_{0}$.

**Step 2.** Test the sign of the product $\phi = \mathrm{sgn}(f(a_{0}))\mathrm{sgn}(f(c_{0}))$. 

*case i.* $\phi < 0$. This means that $f(a_{0})$ and $f(c_{0})$ have opposite signs, which implies that the root lies in the *left* subinterval $[a_{0},c_{0}]$. Bracket the root by taking $c_{0}$ to be the new *right* boundary, i.e., set $a_{1}=a_{0}$ and $b_{1}=c_{0}$.

*case ii.* $\phi > 0$. This means that $f(a_{0})$ and $f(c_{0})$ have the same sign, which implies that the root lies in the *right* subinterval $[c_{0},b_{0}]$. Bracket the root by taking $c_{0}$ to be the new *left* boundary, i.e., set $a_{1}=c_{0}$ and $b_{1}=b_{0}$.

*case iii.* $\phi = 0$. This means that the root happens to be located at $c_{0}$, to within machine precision. STOP!

**Step 3.** Repeat steps 1 and 2 for the new interval $[a_{i},b_{i}]$, where $i$ is the iteration number, incremented by one at the end of each iteration.

Schematically, the process unfolds as follows

\begin{align}
  [a_{0},b_{0}] \rightarrow c_{0} 
  \rightarrow [a_{1},b_{1}] \rightarrow c_{1}
  \rightarrow [a_{2},b_{2}] \rightarrow c_{2}
  \rightarrow \cdots \rightarrow c_{n-1}
  \rightarrow [a_{n},b_{n}] \rightarrow \text{STOP}
\end{align}

where the root has been obtained at the $n$th iteration (or the maximum number of iterations has been reached).

## 2. CODE

In [19]:
import numpy as np
import sys

# bisect method
def bisect(f, a, b, TOL, imax):
    """
    searches for roots of f(x) using the bisection method
    
    INPUT:
    f = function whose roots are being sought 
    a = initial left bracket
    b = initial right bracket
    TOL = allowed tolerance
    imax = maximum number of iterations
    
    OUTPUT:
    
    """
    
    # test initial bracket 
    if np.sign(f(a))*np.sign(f(b)) > 0:
        # if sgn > 0, problem may not be well-defined
        print('ERROR: same initial sign of f(x) at the interval boundaries.')
        print('Are you sure that your initial interval brackets one and only one root?')
        print('Choose a different interval, or use a different root-finding method.')
        sys.exit()

    # initialize iteration
    xLeft = a   # set left bound
    xRight = b  # set right bound
    xMid = xLeft + (xRight - xLeft)/2. #calculate midpoint
    i = 0  # reset iteration number
    
    # iterate search using bisection method
    while i <= imax:
    
        # increment iteration number
        i = i + 1
        
        # perform sign test
        sgn = np.sign(f(xLeft))*np.sign(f(xMid))
        if sgn <= 0:
            # if sgn < 0, root is in left subinterval,
            # make midpt the new right bound
            print('iteration',i,': root is located in left subinterval, between',xLeft,'and',xMid)
            xRight = xMid
        elif sgn > 0:
            # if sgn > 0, root is in right subinterval,
            # make midpt the new left bound
            print('iteration',i,': root is located in right subinterval, between',xMid,'and',xRight)
            xLeft = xMid

        # calculate new midpoint
        xMid = xLeft + (xRight - xLeft)/2.
    
        # check if error is within the allowed tolerance
        delta = xRight - xLeft # current uncertainty is width of interval
        best = xMid            # current best estimate is middle of interval
        err = np.abs(delta/best)       # calculate relative error
        if (err <= TOL):
            print('DONE! root has been located to within the specified tolerance root after',i,'iterations.')
            print('Root is located at',xMid,'+/-',err*best)
            break
            
        # print message if max iteration has been reached
        if i == imax:
            print('STOP! max number of iterations has been reached')

## 3. A nice example

In [20]:
# define function whose roots we are searching for
def myfunc(x):
    return np.cos(x) - x

In [21]:
bisect(myfunc, 0., 2., 1e-8, 100)

iteration 1 : root is located in left subinterval, between 0.0 and 1.0
iteration 2 : root is located in right subinterval, between 0.5 and 1.0
iteration 3 : root is located in left subinterval, between 0.5 and 0.75
iteration 4 : root is located in right subinterval, between 0.625 and 0.75
iteration 5 : root is located in right subinterval, between 0.6875 and 0.75
iteration 6 : root is located in right subinterval, between 0.71875 and 0.75
iteration 7 : root is located in right subinterval, between 0.734375 and 0.75
iteration 8 : root is located in left subinterval, between 0.734375 and 0.7421875
iteration 9 : root is located in right subinterval, between 0.73828125 and 0.7421875
iteration 10 : root is located in left subinterval, between 0.73828125 and 0.740234375
iteration 11 : root is located in left subinterval, between 0.73828125 and 0.7392578125
iteration 12 : root is located in right subinterval, between 0.73876953125 and 0.7392578125
iteration 13 : root is located in right subin

## 4. A not-so-nice example

In [22]:
# define function whose roots we are searching for
def myfunc2(x):
    return np.cos(x)**2 + x

In [23]:
bisect(myfunc2, 0., 6.37, 1e-9, 100)

ERROR: same initial sign of f(x) at the interval boundaries.
Are you sure that your initial interval brackets one and only one root?
Choose a different interval, or use a different root-finding method.


SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 5. Convergence